In [25]:
import config
import json
import requests
import vulture_scraper as vulture
api_key = config.API_key
import time 

In [3]:
## returns data from Movie DB's list of popular movies. updated daily 
def get_popular(api_key):

    url = "https://api.themoviedb.org/3/movie/popular?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [4]:
## returns only the titles from Movie DB's list of popular movies 
def get_popular_titles(api_key):
    data = get_popular(api_key)
    return [x['title'] for x in data['results']]

get_popular_titles(api_key)[:5]

['Joker',
 'Maleficent: Mistress of Evil',
 'Terminator: Dark Fate',
 'Fast & Furious Presents: Hobbs & Shaw',
 'The Lion King']

In [5]:
## returns data from Movie DB's list of top rated movies 
def get_top_rated(api_key):
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [6]:
## returns list of IDs from Movie DB's top rated movies 
def get_top_rated_ids(api_key):
    data = get_top_rated(api_key)
    return [x['id'] for x in data['results']]


In [286]:
## returns full details when given a list of movie IDs
def get_movies(api_key,ids):
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        time.sleep(.5)
        data.append(requests.get(url).json())
    return data

In [241]:
## returns movie data when given an API key and list of movie titles 

def search_movies(api_key, list_of_movies):
    data = []
    for movie in list_of_movies[0:40]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.extend(requests.get(url).json()['results'])
        time.sleep(1)
    #time.sleep(10)
    for movie in list_of_movies[40:]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.extend(requests.get(url).json()['results'])
        time.sleep(1)
    return data


In [184]:
# def search_movies(api_key, list_of_movies):
#     data = []
#     for movie in list_of_movies[0:40]:
#         url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
#         temp = requests.get(url).json()['results']
#         data.extend(temp[0])
#         time.sleep(1)
#     time.sleep(21)
#     for movie in list_of_movies[41:]:
#         url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
#         temp = requests.get(url).json()['results']
#         data.extend(temp[0])
#         time.sleep(1)
#     return data


In [243]:
## search Movie DB for the best movies on Hulu according to Vulture 

hulu_movies = search_movies(api_key, vulture.best_hulu)

In [270]:
netflix_movies = search_movies(api_key, vulture.best_netflix)

In [271]:
amazon_movies = search_movies(api_key, vulture.best_amazon)

In [278]:
print(len(hulu_movies)+len(netflix_movies)+len(amazon_movies))
all_ids = {x['id'] for x in hulu_movies}|{x['id'] for x in netflix_movies}|{x['id'] for x in amazon_movies}
len(set((all_ids)))


2132


1958

In [265]:
get_movies_test = get_movies(api_key,set([x['id'] for x in hulu_movies][0:41]))

In [290]:
all_movie_data = get_movies(api_key,set(all_ids))


KeyboardInterrupt: 

In [333]:
df = pd.DataFrame(all_movie_data)
df.head()
df_rev_budget = df[['imdb_id','title','revenue','budget','tagline','runtime','genres']]
df_rev_budget = df_rev_budget[df_rev_budget['revenue']>0]
df_rev_budget['gross_profit']=df['revenue']-df['budget']
df_rev_budget.sort_values('gross_profit',ascending=False,inplace=True)
df_rev_budget

,imdb_id,title,revenue,budget,tagline,runtime,genres,gross_profit
296,tt4154796,Avengers: Endgame,2797800564,356000000,Part of the journey is the end.,181.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",2441800564
736,tt1825683,Black Panther,1346739107,200000000,Long live the king.,134.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1146739107
802,tt2527336,Star Wars: The Last Jedi,1332459537,200000000,Darkness rises... and light to meet it,152.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",1132459537
1211,tt3606756,Incredibles 2,1241891456,200000000,Back to work.,118.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1041891456
443,tt0298148,Shrek 2,919838758,150000000,Once upon another time...,93.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",769838758
...,...,...,...,...,...,...,...,...
61,tt1046997,Miracle at St. Anna,9323833,45000000,,160.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",-35676167
1479,tt0891592,Street Fighter: The Legend of Chun-Li,12764201,50000000,Some fight for power. Some fight for us.,97.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",-37235799
685,tt0131704,The Adventures of Rocky & Bullwinkle,35134820,76000000,This summer it's not the same old bull.,88.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",-40865180
417,tt0427392,The Invasion,15071514,80000000,Do not trust anyone. Do not show emotion. Do n...,99.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",-64928486


In [334]:
df_rev_budget['genres']=df_rev_budget['genres'].map(lambda x: [y['id'] for y in x])
df_rev_budget.head()

,imdb_id,title,revenue,budget,tagline,runtime,genres,gross_profit
296,tt4154796,Avengers: Endgame,2797800564,356000000,Part of the journey is the end.,181.0,"[12, 878, 28]",2441800564
736,tt1825683,Black Panther,1346739107,200000000,Long live the king.,134.0,"[28, 12, 14, 878]",1146739107
802,tt2527336,Star Wars: The Last Jedi,1332459537,200000000,Darkness rises... and light to meet it,152.0,"[14, 12, 878, 28]",1132459537
1211,tt3606756,Incredibles 2,1241891456,200000000,Back to work.,118.0,"[28, 12, 16, 10751]",1041891456
443,tt0298148,Shrek 2,919838758,150000000,Once upon another time...,93.0,"[12, 16, 35, 10751, 14]",769838758


In [326]:
df_rev_budget.head()

,imdb_id,title,revenue,budget,tagline,runtime,genres,gross_profit
296,tt4154796,Avengers: Endgame,2797800564,356000000,Part of the journey is the end.,181.0,"[Adventure, Science Fiction, Action]",2441800564
736,tt1825683,Black Panther,1346739107,200000000,Long live the king.,134.0,"[Action, Adventure, Fantasy, Science Fiction]",1146739107
802,tt2527336,Star Wars: The Last Jedi,1332459537,200000000,Darkness rises... and light to meet it,152.0,"[Fantasy, Adventure, Science Fiction, Action]",1132459537
1211,tt3606756,Incredibles 2,1241891456,200000000,Back to work.,118.0,"[Action, Adventure, Animation, Family]",1041891456
443,tt0298148,Shrek 2,919838758,150000000,Once upon another time...,93.0,"[Adventure, Animation, Comedy, Family, Fantasy]",769838758


In [339]:
df.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [352]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 25 columns):
adult                    1958 non-null bool
backdrop_path            1136 non-null object
belongs_to_collection    193 non-null object
budget                   1958 non-null int64
genres                   1958 non-null object
homepage                 377 non-null object
id                       1958 non-null int64
imdb_id                  1777 non-null object
original_language        1958 non-null object
original_title           1958 non-null object
overview                 1958 non-null object
popularity               1958 non-null float64
poster_path              1703 non-null object
production_companies     1958 non-null object
production_countries     1958 non-null object
release_date             1958 non-null object
revenue                  1958 non-null int64
runtime                  1719 non-null float64
spoken_languages         1958 non-null object
status                  

False    1958
Name: adult, dtype: int64

In [351]:
for column in ['status']:
    print(df[column].value_counts())

Released           1932
Planned              15
In Production         8
Post Production       2
Rumored               1
Name: status, dtype: int64
